# Introduction to adam_core

In this tutorial, we will work through querying a variety of different external services for the orbits of asteroids. Using their orbits, we'll then venture into propagating them to a desired set of times and we'll generate ephemeris for asteroids for a set of observers for these asteroids. Along the way, we'll touch on some of the underlying types that allow for easy time scale conversions, conversions of orbital elements (including their covariances), among others. 

Contents
- [Installation](#installation)
- [Querying for Orbits](#querying-for-orbits)
    - [Querying JPL Small Body Database (SBDB)](#querying-jpl-small-body-database-sbdb)
    - [Querying JPL Horizons](#querying-horizons)
    - [Querying JPL Scout](#querying-jpl-scout)
    - [Querying ESA NEOCC](#querying-esa-neo-cc)
- [What are these objects?](#what-are-these-objects)
    - [Orbits](#orbits)
    - [Coordinates & Transforms](#coordinates-&-transforms)
    - [Timestamps](#timestamps)
- [Propagating Orbits](#propagating-orbits)
- [Generating Ephemeris](#generating-ephemeris)


## Installation

adam_core is available on PyPI.

```bash
pip install adam-core
```

If you'd like also install the ASSIST propagator (required for propagating orbits with a non 2-body force model), you can install the `test` extras. (We will add `assist` extras soon.)

```bash
pip install adam-core[test]
```

If you'd like to install the plotting and visualization tools, you can install the `plots` extras.

```bash
pip install adam-core[plots]
```

If you're familar with pdm, you can also install adam_core via pdm (assuming you've cloned the repository).

```bash
pdm install -G plots -G dev
```

## Querying for Orbits

### Querying JPL Small Body Database (SBDB)

We can query the JPL Small Body Database (SBDB) for the orbits of objects. These objects are typically defined at a similar epoch to the epoch of the MPC catalog. Our query tool also pulls the full-precision covariance matrix of the state vectors.


In [1]:
from adam_core.orbits.query import query_sbdb

object_ids = ["Ivezic", "Edlu", "Holman", "Deadoklestic"]

orbits_sbdb = query_sbdb(object_ids)
orbits_sbdb

Orbits(size=4)

### Querying JPL Horizons

We can query JPL Horizons (via astroquery) for the orbits of objects at particular times. In adam_core, we have use integer times via the Timestamp class.

In [2]:
from adam_core.orbits.query import query_horizons
from adam_core.time import Timestamp

object_ids = ["Ivezic", "Edlu", "Holman", "Deadoklestic"]
time = Timestamp.from_mjd([60000.0], scale="utc")

orbits_horizons = query_horizons(object_ids, time)
orbits_horizons

Orbits(size=4)

### Querying JPL Scout

JPL's SCOUT tracks NEO candidates that have not yet been designated or added to the official catalog of known objects. SCOUT does not provide the nominal state vectors but instead provides a range of possible state vectors that match the given observations (variants).

In [3]:
from adam_core.orbits.query import query_scout
from adam_core.orbits.query.scout import get_scout_objects

scout_objects = get_scout_objects()
orbits_scout = query_scout(scout_objects.objectName)
orbits_scout

KeyboardInterrupt: 

### Querying ESA NEOCC

We can query the NEOCC (Near Earth Object Coordination Center) of the European Space Agency (ESA) for the orbits of objects (typically NEOs).

In [ ]:
from adam_core.orbits.query import query_neocc

object_ids = ["2024 YR4", "2018 BP1", "2013 RR165"]

orbits_neocc = query_neocc(object_ids)
orbits_neocc

: 

## What are these objects?

We've seen an adam_core Orbits object, a Timestamp object, and even a VariantOrbits object. Let's take a look at what these objects actually are.

### Orbits
Let's start with the Orbits object and venture into the coordinates object attached to it.

In [1]:
orbits = orbits_sbdb
orbits.to_dataframe()

NameError: name 'orbits_sbdb' is not defined

: 

### Coordinates & Transforms

adam_core has support for a variety of different coordinate systems or representations of the state vector. By default, the Orbits class stores Cartesian state vectors.

In [2]:
coords = orbits.coordinates
coords

NameError: name 'orbits' is not defined

: 

In [ ]:
coords.r

: 

In [ ]:
coords.v

: 

In [ ]:
coords.sigma_r

: 

In [ ]:
coords.sigma_v

: 

In [ ]:
# Angular momentum of the orbit in units of au^2 / d
coords.h_mag

: 

Cartesian state vectors are not the easiest to interpret so we've made transformations between the different coordinate system as easy as possible. Other coordinate representations supported are
- SphericalCoordinates ($\rho, \theta, \phi, \dot{\rho}, \dot{\theta}, \dot{\phi}$)
- KeplerianCoordinates (a, e, i, $\Omega$, $\omega$, $M$)
- CometaryCoordinates (q, e, i, $\Omega$, $\omega$, $t_p$)

Notice that in each of the following transformations, we've also transformed the covariance matrices to the new representation.

In [ ]:
keplerian_coordinates = orbits.coordinates.to_keplerian()
keplerian_coordinates.to_dataframe()

: 

In [ ]:
cometary_coordinates = orbits.coordinates.to_cometary()
cometary_coordinates.to_dataframe()

: 

In [ ]:
spherical_coordinates = orbits.coordinates.to_spherical()
spherical_coordinates.to_dataframe()


: 

For a more general transformation, which can include rotations to different frames of reference, translations to different origins, etc., we can use the `transform_coordinates` function. Again, notice that the covariance matrices are transformed along with the state vectors.

In [ ]:
from adam_core.coordinates import transform_coordinates
from adam_core.coordinates import OriginCodes, KeplerianCoordinates

coords_geo_keplerian = transform_coordinates(
    orbits.coordinates,
    representation_out=KeplerianCoordinates,
    frame_out="equatorial",
    origin_out=OriginCodes.EARTH
)

coords_geo_keplerian.to_dataframe()

: 

## Timestamps

Coordinates also contain a time at which they are defined. We've created a custom `Timestamp` class that stores times as two 64-bit integers (days and nanoseconds). A driving motivation for this is that matching on integers is a lot easier than trying to match on floats. For example, trying to join an exposures table with an ephemeris table on float values (such as MJDs) does not always guarantee the desired match especially if the floating point values are derived from different sources (propagator vs catalog). Doing this with integers is a lot easier and we've defined functions in the cases where the matches are not exact. 

In [4]:
from adam_core.time import Timestamp

time = Timestamp.from_kwargs(
    days=[60000, 60000],
    nanos=[1e6, 1e6 + 0.5e6],
    scale="tai"
)

time.to_dataframe()

,days,nanos
0,60000,1000000
1,60000,1500000


In [5]:
time[1].difference(time[0])

(<pyarrow.lib.Int64Array object at 0x126a970a0>
 [
   0
 ],
 [
   500000
 ])

In [10]:
print(time[1].equals(time[0], precision="ms"))

print(time[1].equals(time[0], precision="us"))


[
  true
]
[
  false
]


In [15]:
astropy_time = time.to_astropy()

astropy_time.isot

array(['2023-02-25T00:00:00.001', '2023-02-25T00:00:00.001'], dtype='<U23')

In [24]:
from astropy.time import Time

astropy_time = Time(60000.0, format="mjd", scale="tai")
time_from_astropy = Timestamp.from_astropy(astropy_time)

print(time_from_astropy.to_dataframe())
print(time_from_astropy.scale)

    days  nanos
0  60000      0
tai


: 

: 

## Propagating Orbits

adam_core has support for propagating orbits using a two different propagators (ASSIST and PYOORB). We recommend using the ASSIST propagator for most use cases as it has the more robust force model (thank you, Matt Holman!).

You can also add your own propagators by subclassing the `Propagator` class and implementing the `_propagate_orbits` method. The base `Propagator` class handles the multiprocessing (via `ray`) of the propagation, so you only need to implement the core logic of how to propagate a chunk of orbits. Likewise, if your propagator can handle ephemeris generation you can subclass the `EphemerisMixin` class and implement the `_generate_ephemeris` method. If your propagator can also handle collision (or impact) detection you can subclass the `ImpactMixin` class and implement the `_detect_impacts` method.

Here's an example of propagating orbits using the ASSIST propagator.

In [ ]:
import numpy as np
from adam_assist import ASSISTPropagator

propagator = ASSISTPropagator()

# Create a set of times to propagate the orbits to
times = Timestamp.from_mjd(np.arange(60000, 61000, 5))

# Note: show parallel processing and covariance propagation
propagated_orbits = propagator.propagate_orbits(orbits, times)
propagated_orbits.to_dataframe()

: 

Now that we have been introduced to the Orbits and Propagator classes. Let's just quickly preview our input orbits:


In [ ]:
orbits.preview(propagator)

: 

## Generating Ephemeris

Let's continue using the ASSIST propagator to generate ephemeris for a set of observers for the given set of orbits.

In [ ]:
from adam_core.observers import Observers


# Note: we gather the observer's heliocentric state vectors are the given times which will allow us to also support non-Earth observers (like spacecraft or other asteroids)
# Note note: how does this work? -- Data packages
observers = Observers.from_code("X05", times)
observers.to_dataframe()

: 

In [67]:
ephemeris = propagator.generate_ephemeris(orbits, observers, max_processes=10)
ephemeris.to_dataframe()

: 